In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import dask
import os, sys
import glob
import zarr
from joblib import Parallel, delayed
import os
import dask.array as da

sys.path.append('/')
from libraries import *

dates = pd.date_range(start='2011-01-01T00', end='2020-12-31T23', freq='h')
wind_speed_zarr_store = '/media/harish/External_3/CERRA_wind_profiles_and_Chebyshev_coefficients/CERRA_height_level_winds.zarr'
Cheybshev_zarr_store = '/media/harish/External_3/CERRA_wind_profiles_and_Chebyshev_coefficients/CERRA_Chebyshev_coefficients.zarr'

In [2]:
print("Starting parallel computing...")
import dask.distributed as dd
cluster = dd.LocalCluster(n_workers=48,threads_per_worker=1,memory_limit='4GB',dashboard_address='8787')
# Connect to the cluster
client = dd.Client(cluster)
print(client)

Starting parallel computing...
<Client: 'tcp://127.0.0.1:44017' processes=48 threads=48, memory=178.81 GiB>


In [ ]:
# Open the Zarr store
wind_speed_store = zarr.open(wind_speed_zarr_store, mode="r")

# Select dimensions
time_dim = wind_speed_store["time"]
height_dim = wind_speed_store["heightAboveGround"]
wind_speed_store.wind_speed[0:31*24,:,100,100].shape

In [13]:
# Load the Zarr dataset and subset for one time step
ds = xr.open_zarr(wind_speed_zarr_store, chunks={'heightAboveGround':12}).wind_speed.sel(time=slice('2020-01-01T00','2020-01-31T23'))
ds

<xarray.DataArray 'wind_speed' (time: 744, heightAboveGround: 12, y: 1069,
                                x: 1069)>
dask.array<getitem, shape=(744, 12, 1069, 1069), dtype=float32, chunksize=(1, 12, 1069, 1069), chunktype=numpy.ndarray>
Coordinates:
  * heightAboveGround  (heightAboveGround) float64 10.0 15.0 ... 400.0 500.0
  * time               (time) datetime64[ns] 2020-01-01 ... 2020-01-31T23:00:00
Dimensions without coordinates: y, x
Attributes: (12/33)
    GRIB_DxInMetres:                          5500.0
    GRIB_DyInMetres:                          5500.0
    GRIB_LaDInDegrees:                        50.0
    GRIB_Latin1InDegrees:                     50.0
    GRIB_Latin2InDegrees:                     50.0
    GRIB_LoVInDegrees:                        8.0
    ...                                       ...
    GRIB_stepUnits:                           1
    GRIB_typeOfLevel:                         heightAboveGround
    GRIB_units:                               m s**-1
    long_name:                                Wind speed
    standard_name:                            unknown
    units:                                    m s**-1

In [14]:
chebyshev_vec(ds, dim="heightAboveGround").compute()

<xarray.DataArray 'Chebyshev_coefficients' (time: 744, y: 1069, x: 1069,
                                            coeff: 5)>
array([[[[ 1.11315419e+01,  1.82704133e+00, -1.54536365e+00,
           2.84394936e-01, -9.80479210e-02],
         [ 1.11646841e+01,  1.85473026e+00, -1.54550030e+00,
           2.86554330e-01, -1.01979645e-01],
         [ 1.11704742e+01,  1.87857224e+00, -1.53493515e+00,
           2.78974400e-01, -1.09862000e-01],
         ...,
         [ 8.95296290e+00,  1.43926136e+00, -1.91932822e+00,
           6.14468755e-01, -6.86908219e-02],
         [ 8.73639856e+00,  1.36895014e+00, -1.84314488e+00,
           6.24844149e-01, -8.95190553e-02],
         [ 8.47241108e+00,  1.31179800e+00, -1.75668276e+00,
           6.35391030e-01, -1.08395118e-01]],

        [[ 1.11631039e+01,  1.84865424e+00, -1.54809296e+00,
           2.84522089e-01, -9.98901887e-02],
         [ 1.11666226e+01,  1.86731912e+00, -1.54103764e+00,
           2.81401868e-01, -1.02727424e-01],
         [ 1.11768133e+01,  1.89611639e+00, -1.52598297e+00,
           2.79097233e-01, -1.12820733e-01],
...
         [ 1.14657147e+01,  1.46783066e+00, -2.03278893e+00,
           8.36923055e-01, -1.65068558e-01],
         [ 1.13766861e+01,  1.51623797e+00, -2.02437313e+00,
           8.13860299e-01, -1.39327971e-01],
         [ 1.12320247e+01,  1.58360281e+00, -2.02027014e+00,
           7.83998140e-01, -1.13898178e-01]],

        [[ 9.89177966e+00,  2.17151048e-01, -2.38298500e-01,
           1.60288900e-01, -1.03193690e-01],
         [ 1.00200849e+01,  2.49158678e-01, -2.45785597e-01,
           1.59118989e-01, -1.06837167e-01],
         [ 1.01319312e+01,  2.80166763e-01, -2.57470741e-01,
           1.57437762e-01, -1.14529213e-01],
         ...,
         [ 1.13115217e+01,  1.45552921e+00, -1.98752893e+00,
           8.43329412e-01, -1.84594597e-01],
         [ 1.12212693e+01,  1.51912712e+00, -1.98626648e+00,
           8.17720953e-01, -1.63005167e-01],
         [ 1.10547107e+01,  1.57749748e+00, -1.97784497e+00,
           7.88700905e-01, -1.41646108e-01]]]])
Coordinates:
  * time     (time) datetime64[ns] 2020-01-01 ... 2020-01-31T23:00:00
Dimensions without coordinates: y, x, coeff